In [1]:
import pandas as pd

df = pd.read_csv('/Users/suryanshpatel/Projects/Directed Readings/Technical/src/data/Finaldata_combined.csv')
df.head()

,Subject,Trial,Sweep,Label,Timestamp,Full_Path_RGB,Full_Path_Depth
0,2,2,10,0,03-32-54-8112,/Users/suryanshpatel/Projects/Directed Reading...,/Users/suryanshpatel/Projects/Directed Reading...
1,2,2,10,0,03-32-55-3457,/Users/suryanshpatel/Projects/Directed Reading...,/Users/suryanshpatel/Projects/Directed Reading...
2,2,2,10,0,03-32-55-4113,/Users/suryanshpatel/Projects/Directed Reading...,/Users/suryanshpatel/Projects/Directed Reading...
3,2,2,10,0,03-32-55-4463,/Users/suryanshpatel/Projects/Directed Reading...,/Users/suryanshpatel/Projects/Directed Reading...
4,2,2,10,0,03-32-54-5795,/Users/suryanshpatel/Projects/Directed Reading...,/Users/suryanshpatel/Projects/Directed Reading...


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54385 entries, 0 to 54384
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Subject          54385 non-null  int64 
 1   Trial            54385 non-null  int64 
 2   Sweep            54385 non-null  int64 
 3   Label            54385 non-null  int64 
 4   Timestamp        54385 non-null  object
 5   Full_Path_RGB    54385 non-null  object
 6   Full_Path_Depth  54385 non-null  object
dtypes: int64(4), object(3)
memory usage: 2.9+ MB


In [3]:
df.describe()

,Subject,Trial,Sweep,Label
count,54385.000000,54385.000000,54385.000000,54385.00000
mean,10.307162,1.558757,20.533640,1.25222
std,5.819282,0.496540,11.388181,1.48163
min,1.000000,1.000000,1.000000,0.00000
25%,5.000000,1.000000,11.000000,0.00000
50%,10.000000,2.000000,21.000000,0.00000
75%,15.000000,2.000000,30.000000,3.00000
max,20.000000,2.000000,40.000000,4.00000


In [4]:
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Initialize MediaPipe modules
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

# Load CSV containing image paths
csv_path = "/Users/suryanshpatel/Projects/Directed Readings/Technical/src/data/Finaldata-part1.csv"
df = pd.read_csv(csv_path)

# Define augmentations for RGB images
image_transform = A.Compose([
    A.Resize(224, 224),  # Resize to match CNN models
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # Standard normalization
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    ToTensorV2()
])

# Function to process RGB images
def preprocess_rgb(image_path):
    image = cv2.imread(image_path)  # Read image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    image = image_transform(image=image)["image"]  # Apply augmentations
    return image

# Function to process depth images
def preprocess_depth(image_path):
    depth_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read depth image in grayscale
    depth_image = cv2.resize(depth_image, (224, 224))  # Resize
    depth_image = cv2.GaussianBlur(depth_image, (5, 5), 0)  # Noise reduction
    depth_image = depth_image / 255.0  # Normalize to [0,1]
    return np.expand_dims(depth_image, axis=0)  # Add channel dimension

# Function to extract face and posture features using MediaPipe
def extract_mediapipe_features(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Face Mesh Detection
    face_landmarks_list = []
    with mp_face_mesh.FaceMesh(static_image_mode=True) as face_mesh:
        results = face_mesh.process(image_rgb)
        if results.multi_face_landmarks:
            for landmark in results.multi_face_landmarks[0].landmark:
                face_landmarks_list.append((landmark.x, landmark.y, landmark.z))

    # Pose Detection
    pose_landmarks_list = []
    with mp_pose.Pose(static_image_mode=True) as pose:
        results = pose.process(image_rgb)
        if results.pose_landmarks:
            for landmark in results.pose_landmarks.landmark:
                pose_landmarks_list.append((landmark.x, landmark.y, landmark.z))

    # Flatten lists into feature vectors
    face_features = np.array(face_landmarks_list).flatten() if face_landmarks_list else np.zeros(468 * 3)
    pose_features = np.array(pose_landmarks_list).flatten() if pose_landmarks_list else np.zeros(33 * 3)

    return np.concatenate((face_features, pose_features))

# Process dataset
processed_data = []
for _, row in df.iterrows():
    rgb_path = row['RGB_Path']
    depth_path = row['Depth_Path']
    label = row['Pain_Level']  # Adjust based on your dataset labels

    # Process images and extract features
    rgb_image = preprocess_rgb(rgb_path)
    depth_image = preprocess_depth(depth_path)
    mediapipe_features = extract_mediapipe_features(rgb_path)

    # Append to list (Flattening for model input)
    processed_data.append({
        "RGB": rgb_image.numpy(),
        "Depth": depth_image,
        "Pose_Features": mediapipe_features,
        "Label": label
    })

# Convert to DataFrame or save as numpy array
import pickle
with open("processed_mint_pain.pkl", "wb") as f:
    pickle.dump(processed_data, f)

print("✅ Preprocessing Complete. Saved dataset!")


ModuleNotFoundError: No module named 'numpy.rec'